In [72]:
import os
import urllib
import json

import torch
from PIL import Image
import numpy as np
import requests
from torchvision import transforms

In [73]:
class TensorToJson(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
def tensor_to_json(tensor):
    array = tensor.numpy()
    return json.dumps(array, cls=TensorToJson)

In [97]:
INFERENCE_ENDPOINT = os.environ.get("INFERENCE_ENDPOINT")


transform_test = transforms.Compose([
    transforms.Resize((600, 600), Image.BILINEAR),
    transforms.CenterCrop((448, 448)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])


# Get image and prepare for model
url = 'https://upload.wikimedia.org/wikipedia/commons/d/d8/Cardinalis_cardinalis_-Columbus%2C_Ohio%2C_USA-male-8_%281%29.jpg'
img = Image.open(urllib.request.urlopen(url))
scaled_img = transform_test(img)
torch_images = scaled_img.unsqueeze(0)
image = torch_images.numpy()
im_json = image.tolist()

# Build the request body
payload = {
    "inputs": [
        {
            # The ONNX model requires this name
            "name": "torch_images",
            "shape": [1, 3, 448, 448],
            "datatype": "FP32",
            "data": im_json
        }
    ]
}

# Send request
response = requests.post(INFERENCE_ENDPOINT, json=payload)
result = response.json()

# Get class
logits = result["outputs"][0]
pred_id = np.argmax(logits["data"])
print('bird class idx :', pred_id)

with open('bird_classes.json', 'r') as f:
    bird_classes = json.load(f)
    
print('bird class:', bird_classes[pred_id])

bird class idx : 16
bird class: 017.Cardinal
